In [1]:
# Importando bibliotecas necessárias para a construção de grafos
import netpixi
from netpixi.integration.gt import *
import pandas as pd
import numpy as np
import csv
import ast

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [2]:
df = pd.read_csv('./Video_Games_Sales_as_at_22_Dec_2016.csv')
df.dropna(subset=['Publisher'], inplace=True)
df

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.groupby('Genre')['Name'].count().sort_values(ascending=False)

Genre
Action          3364
Sports          2345
Misc            1723
Role-Playing    1499
Shooter         1321
Adventure       1299
Racing          1248
Platform         887
Simulation       870
Fighting         847
Strategy         680
Puzzle           580
Name: Name, dtype: int64

In [4]:
df_grouped_Genre = df.groupby(['Genre']).agg({'Global_Sales': np.mean})
df_grouped_Genre.sort_values('Global_Sales', ascending=False)

,Global_Sales
Genre,
Platform,0.933033
Shooter,0.797048
Role-Playing,0.623316
Racing,0.584046
Sports,0.567343
Fighting,0.524522
Action,0.518564
Misc,0.461619
Simulation,0.448483


In [5]:
sports_mean_sales = df_grouped_Genre.loc['Sports']['Global_Sales']
action_mean_sales = df_grouped_Genre.loc['Action']['Global_Sales'] 
rpg_mean_sales = df_grouped_Genre.loc['Role-Playing']['Global_Sales'] 

In [6]:
def segregate_genre_to_list(df, genre, df_list, threshold=None):
    seg_df = df[df['Genre'] == genre]
    if threshold:
        seg_df = seg_df[seg_df['Global_Sales'] > threshold]
    df_list.append(seg_df)

In [7]:
sorted_years = df['Year_of_Release'].unique()
sorted_years.sort()
print(sorted_years)

[1980. 1981. 1982. 1983. 1984. 1985. 1986. 1987. 1988. 1989. 1990. 1991.
 1992. 1993. 1994. 1995. 1996. 1997. 1998. 1999. 2000. 2001. 2002. 2003.
 2004. 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013. 2014. 2015.
 2016. 2017. 2020.   nan]


In [8]:
df_list = []
sport_df_list = []
action_df_list = []
rpg_df_list = []
margem = 4
for i in range(0, len(sorted_years), margem):
    print(i)
    df_list.append(df[df['Year_of_Release'].isin(sorted_years[i:i+margem])])


0
4
8
12
16
20
24
28
32
36


In [9]:
df_list[0]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
89,Pac-Man,2600,1982.0,Puzzle,Atari,7.28,0.45,0.00,0.08,7.81,NaN,NaN,NaN,NaN,NaN,NaN
240,Pitfall!,2600,1981.0,Platform,Activision,4.21,0.24,0.00,0.05,4.50,NaN,NaN,NaN,NaN,NaN,NaN
262,Asteroids,2600,1980.0,Shooter,Atari,4.00,0.26,0.00,0.05,4.31,NaN,NaN,NaN,NaN,NaN,NaN
426,Baseball,NES,1983.0,Sports,Nintendo,0.73,0.10,2.35,0.02,3.20,NaN,NaN,NaN,NaN,NaN,NaN
546,Missile Command,2600,1980.0,Shooter,Atari,2.56,0.17,0.00,0.03,2.76,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8458,Death Trap,2600,1982.0,Action,Avalon Interactive,0.15,0.01,0.00,0.00,0.17,NaN,NaN,NaN,NaN,NaN,NaN
8459,Armor Ambush,2600,1981.0,Action,Mattel Interactive,0.15,0.01,0.00,0.00,0.17,NaN,NaN,NaN,NaN,NaN,NaN
8740,Pressure Cooker,2600,1982.0,Action,Activision,0.14,0.01,0.00,0.00,0.15,NaN,NaN,NaN,NaN,NaN,NaN
9096,Krull,2600,1982.0,Action,Atari,0.13,0.01,0.00,0.00,0.14,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
publisher_platforms = []
for db in df_list:
    publisher_platforms.append(db.groupby('Platform')['Publisher'].unique().to_dict())
publisher_platforms

[{'2600': array(['Atari', 'Activision', 'Parker Bros.', 'Imagic', 'Coleco',
         'Quelle', 'Mystique', '20th Century Fox Video Games',
         'Men-A-Vision', 'Tigervision', 'Mattel Interactive',
         'Universal Gamex', 'Wizard Video Games', 'CPG Products',
         'Answer Software', 'Sega', 'Data Age', 'ITT Family Games',
         'Starpath Corp.', 'CBS Electronics', 'Unknown', 'Taito', 'Bomb',
         'Avalon Interactive'], dtype=object),
  'NES': array(['Nintendo'], dtype=object)},
 {'2600': array(['Activision', 'Atari'], dtype=object),
  'DS': array(['Mycom'], dtype=object),
  'NES': array(['Nintendo', 'Namco Bandai Games', 'Enix Corporation', 'Capcom',
         'Hudson Soft', 'Konami Digital Entertainment', 'SquareSoft'],
        dtype=object),
  'PC': array(['Activision'], dtype=object)},
 {'2600': array(['Activision', 'Atari'], dtype=object),
  'GB': array(['Nintendo', 'Capcom', 'SquareSoft', 'Namco Bandai Games',
         'Konami Digital Entertainment'], dtype=object

In [11]:
publishers = df["Publisher"].unique()
publishers

array(['Nintendo', 'Microsoft Game Studios', 'Take-Two Interactive',
       'Sony Computer Entertainment', 'Activision', 'Ubisoft',
       'Bethesda Softworks', 'Electronic Arts', 'Sega', 'SquareSoft',
       'Atari', '505 Games', 'Capcom', 'GT Interactive',
       'Konami Digital Entertainment', 'Square Enix',
       'Sony Computer Entertainment Europe', 'Virgin Interactive',
       'LucasArts', 'Warner Bros. Interactive Entertainment',
       'Universal Interactive', 'Eidos Interactive', 'RedOctane',
       'Vivendi Games', 'Enix Corporation', 'Namco Bandai Games',
       'Palcom', 'Hasbro Interactive', 'THQ', 'Fox Interactive',
       'Acclaim Entertainment', 'MTV Games', 'Disney Interactive Studios',
       'Codemasters', 'Majesco Entertainment', 'Red Orb', 'Level 5',
       'Arena Entertainment', 'Midway Games', 'JVC', 'Deep Silver',
       'NCSoft', '989 Studios', 'UEP Systems', 'Parker Bros.', 'Maxis',
       'Imagic', 'Tecmo Koei', 'ASCII Entertainment', 'Valve Software',
     

In [12]:
df_list[1]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
9,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31,NaN,NaN,NaN,NaN,NaN,NaN
124,The Legend of Zelda,NES,1986.0,Action,Nintendo,3.74,0.93,1.69,0.14,6.51,NaN,NaN,NaN,NaN,NaN,NaN
252,Zelda II: The Adventure of Link,NES,1987.0,Adventure,Nintendo,2.19,0.50,1.61,0.08,4.38,NaN,NaN,NaN,NaN,NaN,NaN
281,Excitebike,NES,1984.0,Racing,Nintendo,2.04,0.48,1.57,0.07,4.16,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6557,Desert Falcon,2600,1987.0,Shooter,Atari,0.24,0.01,0.00,0.00,0.26,NaN,NaN,NaN,NaN,NaN,NaN
11748,Super Football,2600,1987.0,Sports,Atari,0.07,0.00,0.00,0.00,0.08,NaN,NaN,NaN,NaN,NaN,NaN
11778,Super Baseball,2600,1987.0,Sports,Atari,0.07,0.00,0.00,0.00,0.07,NaN,NaN,NaN,NaN,NaN,NaN
14472,Alter Ego,PC,1985.0,Simulation,Activision,0.00,0.03,0.00,0.01,0.03,59.0,9.0,5.8,19.0,"Viva Media, Viva Media, LLC",T


In [13]:
df['Name'].loc[df["Publisher"] == 'Nintendo'].nunique() 

671

In [14]:
action_graph = Graph(directed=False) # não-dirigido
action_graph.add_vp('Publisher')
action_graph.add_vp('Games_released')
action_graph.add_ep('strength')

for pub in publishers:
    action_graph.add_vertex(pub)
    action_graph.get_vertex(pub)['Publisher'] = pub
    num_games = df['Name'].loc[df["Publisher"] == pub].nunique() 
    action_graph.get_vertex(pub)['Games_released'] = num_games

# for df in df_list:
#     for pub in publishers:
#         for pub2 in publishers:
#             if pub != pub2 and not action_graph.has_edge(pub, pub2):
#                 action_graph.add_edge(pub, pub2)

In [15]:
for platform_dict in publisher_platforms:
    for platform, publishers in platform_dict.items():
        for i in range(len(publishers)):
            if i != len(publishers)-1:
                next_vertex = i+1
                while(next_vertex < len(publishers)):
                    if(publishers[i] != publishers[next_vertex] and not action_graph.has_edge(publishers[i], publishers[next_vertex])):
                        action_graph.add_edge(publishers[i], publishers[next_vertex])
                        action_graph.get_edge(publishers[i], publishers[next_vertex])['strength'] = 1
                        next_vertex += 1
                    else:
                        action_graph.get_edge(publishers[i], publishers[next_vertex])['strength'] += 1
                        next_vertex += 1

In [16]:
print('número de vértices de g:', action_graph.num_edges())

número de vértices de g: 42765


In [17]:
gt_save(action_graph, 'action_graph.net.gz')

In [18]:
g = gt_load('action_graph.net.gz')
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'action_sfdp.net.gz')

In [28]:
r = netpixi.render('action_sfdp.net.gz', infinite=True);

In [29]:
r.vertex_scale('Games_released', 20, 50)
r.edge_scale('strength', 0.3, 4)
r.vertex_default(color=0xff8674, kscale = 4, bwidth=3, bcolor=0x000000)
r.edge_default(curve1=1, curve2=1)
r.vertex_set_key('Publisher')